# Hacking units into BPD function

Can we get `pint` units in the BPD function?

## What is `pint`?

[`pint`](https://pint.readthedocs.io/en/stable/) is a package for handling physical units. It's really useful — you can compute quantities in any units, and easily convert to new ones. First you'll need to install it:

    pip install pint

Let's have a look. First we set up the 'registry':

In [1]:
from pint import UnitRegistry

u = UnitRegistry()

`pint` lets us have units for everything, and it will magically do conversions when needed.

In [2]:
rho = 1000 * u.kg / u.m**3
g = 9.81 * u.m / u.s**2
i = 3.281 * u.ft  # Uh oh! Mixed units!

p1 = 1 * u.atm
p2 = rho * g * i  # in Pa

p = p1 + p2
p

1.0968218971428572 <Unit('standard_atmosphere')>

We can convert quantities:

In [3]:
p.to('MPa')

0.11113547872799999 <Unit('megapascal')>

NumPy can use `pint` too:

In [4]:
import numpy as np

np.arange(10) * p

Magnitude,[0.0 1.0968218971428572 2.1936437942857143 3.2904656914285715 4.387287588571429 5.484109485714286 6.580931382857143 7.67775328 8.774575177142857 9.871397074285714]
Units,standard_atmosphere


However, not all libraries can use it — `iapws` doesn't like it:

In [5]:
import iapws

p = 0.1 * u.MPa

iapws.iapws97._TSat_P(p)

# This gives an error...

ValueError: Cannot compare Quantity and <class 'float'>

But we can get the magnitude:

In [6]:
p.magnitude  # or p.m (but this looks a bit like 'metres')

0.1

Another example:

In [7]:
pressure = (1.01325 * u.bar).to('MPa')
tsat = iapws.iapws97._TSat_P(pressure.magnitude)
rho = iapws.iapws97.IAPWS97_PT(pressure.magnitude, tsat)
rho.rho * u.kg / u.m**3

958.3727293380052 <Unit('kilogram / meter ** 3')>

## Using `pint` in the BPD function

In [8]:
import pint
from pint import UnitRegistry
from collections import namedtuple

def hydrostatic_bpdc(depth, p0=None, method="iapws", units='common'):
    '''
    Function to calculate the hydrostatic boiling point depth curve, similar to:
    Haas Jr., J.L., 1971. The effect of salinity on the maximum thermal gradient 
    of a hydrothermal system at hydrostatic pressure. Econ. Geol. 66, 940–946.

    Parameters
    ----------
    depth : array-like
        An array of depths at which to evaluate the function.
    p : float, optional
        surface pressure of well. The default is 1.01325 bar.
    method : string
        Method to calculate brine density. The default is "iapws".
    units : string or tuple
        String units for depth, pressure, temperature, and density
        respectively. The following are allowed:
        - 'common' to use metres, bar, deg Celsius, and kg/m**3.
        - 'SI' to use metres, Pascal, Kelvin, and kg/m**3.
        - 'Imperial' to use feet, psia, deg Fahrenheit, and g/cm**3.
        - tuple like ('m', 'atm', 'degC', 'g/cm**3') for custom units.

    Returns
    -------
    namedtuple
    '''
    u = UnitRegistry()

    g = 9.81 * u.m / u.s**2

    # Assign units to everything.
    if units == 'SI':
        u_d, u_p, u_t, u_rho = u.m, u.pa, u.K, u.kg/u.m**3
    if units == 'common':
        u_d, u_p, u_t, u_rho = u.m, u.bar, u('degC'), u.kg/u.m**3
    elif units == 'Imperial':
        u_d, u_p, u_t, u_rho = u.ft, u.psi, u('degF'), u.g/u.cm**3
    else:
        u_d, u_p, u_t, u_rho = list(map(u, units))
    
    # Override units with pint Quantity, if possible.
    # And compute the diff array.
    if isinstance(depth, pint.Quantity):
        u_d = depth.units
    else:
        depth = np.asanyarray(depth) * u_d
    depth_diff = np.diff(depth)
                
    # Override units with pint Quantity, if possible.
    # And deal with not getting a p0 pressure.
    if isinstance(p0, pint.Quantity):
        u_p = p0.units
    elif p0 is None:
        p0 = 101325 * u_p
    else:
        p0 = p0 * u_p  
    p0 = p0.to('MPa')  # Use MPa for calculations (required by IAPWS).

    # Compute using IAPWS option.
    if method == "iapws":
        pressure = np.atleast_1d(p0)
        tsat = iapws.iapws97._TSat_P(p0.m) * u.K
        rho = iapws.iapws97.IAPWS97_PT(p0.m, tsat.m).rho * u.kg / u.m**3
        density = np.atleast_1d(rho)
        
        for i in depth_diff:
            # Calculate new pressure for this step.
            new_p = pressure[-1] + rho * g * i
            pressure = np.append(pressure, new_p)  # Has units.
            
            # Calculate new temperature for this step.
            t = iapws.iapws97._TSat_P(new_p.m) * u.K
            tsat = np.append(tsat, t)
            
            # Calculate new density for next step.
            rho = iapws.iapws97.IAPWS97_PT(new_p.m, t.m).rho * u.kg / u.m**3
            density = np.append(density, rho)
        
        # Finalize units.
        pressure = pressure.to(u_p)
        tsat = tsat.to(u_t)
        density = density.to(u_rho)
        
    # Return a namedtuple to the user to retain units.
    BPD = namedtuple('BPD', ['depth', 'pressure', 'tsat', 'rho'])
    return BPD(depth, pressure, tsat, density)

## Test it out!

Read dataset and compute gauge pressure.

In [9]:
import pandas as pd

url = "https://raw.githubusercontent.com/ICWallis/T21-Tutorial-WellTestAnalysis/main/Data-Temp-Heating37days.csv"

df = pd.read_csv(url)

df['pres_bara'] = df['pres_barg'] - (1 * u.atm).to('bar').magnitude

In [10]:
df

,depth_m,whp_barg,pres_barg,temp_degC,pres_bara
0,0.0,4.019268,3.929268,10.48,2.916018
1,10.5,4.019268,3.929268,10.86,2.916018
2,20.7,4.019268,3.929268,11.63,2.916018
3,30.4,4.019268,3.939268,12.25,2.926018
4,40.3,4.019268,3.949268,12.77,2.936018
...,...,...,...,...,...
89,890.8,4.019268,48.759268,230.79,47.746018
90,900.7,4.019268,49.549268,230.82,48.536018
91,910.5,4.019268,50.359268,230.81,49.346018
92,920.4,4.079268,51.169268,230.56,50.156018


Now we can calculate the BPD curve.

In [11]:
hyd = hydrostatic_bpdc(depth=df['depth_m'], 
                       p0=df["pres_bara"].min(),
                       method='iapws',
                       units=['m', 'bar', 'degC', 'kg/m**3'],
                      )

hyd

BPD(depth=<Quantity([  0.   10.5  20.7  30.4  40.3  50.1  61.1  70.8  80.5  90.3 100.2 110.7
 120.4 130.5 140.5 150.7 160.7 170.3 181.1 190.7 200.1 210.4 220.2 230.2
 240.2 250.2 260.3 270.2 280.1 291.2 301.1 311.1 320.9 330.4 340.1 350.9
 360.2 370.8 380.4 390.1 400.1 411.  420.7 430.2 441.  450.4 460.9 470.8
 480.9 490.8 500.9 510.8 520.4 530.1 541.  550.5 560.8 570.3 580.2 590.
 600.  610.2 620.1 631.  640.7 651.1 660.8 670.7 680.6 690.6 700.3 711.1
 720.7 730.3 741.1 750.5 760.8 770.3 780.1 791.2 801.2 811.2 820.1 831.
 840.7 850.4 860.1 870.8 880.  890.8 900.7 910.5 920.4 926.4], 'meter')>, pressure=<Quantity([ 2.91601773  3.87668905  4.80116418  5.67348125  6.5579369   7.42813206
  8.39946787  9.2510689  10.0986162  10.95104422 11.8084436  12.71401007
 13.54703507 14.41114783 15.26347172 16.12969803 16.97590523 17.7855023
 18.69340625 19.49762435 20.28271336 21.14048631 21.95409333 22.78191071
 23.60734047 24.43043477 25.25945061 26.06981016 26.87801647 27.78181742
 28.58558458 2

## Get a `pandas.DataFrame`

We can carry the units into a `pandas.DataFrame`:

In [12]:
df = pd.DataFrame(hyd).T  # I don't really know why we have to transpose here.
df.columns = columns=hyd._fields
df.head()

,depth,pressure,tsat,rho
0,0.0 meter,2.916017731307572 bar,132.55965119728978 degree_Celsius,932.6453227768354 kilogram / meter ** 3
1,10.5 meter,3.8766890460338517 bar,142.48713946105977 degree_Celsius,923.9023170397114 kilogram / meter ** 3
2,20.7 meter,4.801164182510128 bar,150.31357792517576 degree_Celsius,916.7135032819888 kilogram / meter ** 3
3,30.4 meter,5.67348125082817 bar,156.65782181639696 degree_Celsius,910.6927049160709 kilogram / meter ** 3
4,40.3 meter,6.557936898915608 bar,162.3391734790648 degree_Celsius,905.1521411464424 kilogram / meter ** 3


Or we can throw the units out:

In [13]:
df = pd.DataFrame(hyd._asdict())
df.head()

/Users/matt/opt/miniconda3/envs/geohip/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1302: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  v = np.array(v, copy=False)
/Users/matt/opt/miniconda3/envs/geohip/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1672: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  subarr = np.array(values, dtype=dtype, copy=copy)


,depth,pressure,tsat,rho
0,0.0,2.916018,132.559651,932.645323
1,10.5,3.876689,142.487139,923.902317
2,20.7,4.801164,150.313578,916.713503
3,30.4,5.673481,156.657822,910.692705
4,40.3,6.557937,162.339173,905.152141
